<h1><center><i>'Let's Netflix and Chill'</i></center></h1>

In [ ]:
!pip install imdbpy

In [ ]:
import pandas as pd
import seaborn as sns
from datetime import date
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import __version__
from collections import Counter
from imdb import IMDb
import warnings



init_notebook_mode(connected=True)
cf.go_offline()
ia = IMDb()
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
iurl='https://github.com/imamanmehrotra/Images_for_projects/raw/main/Netflix.jpeg'
from IPython.display import Image
Image(iurl)

In [ ]:
url='https://raw.githubusercontent.com/imamanmehrotra/Datasets/main/netflix.csv'
df=pd.read_csv(url)
df.head(10)

## Data Cleaning and Initial Level Analysis

In [ ]:
df.info()

## Out of 11 Features above only 1 is having INT data type and every other feature is of object type

In [ ]:
print('Rows:',df.shape[0])
print('\n')
print('Columns:',df.shape[1])
print('\n')
print('Features/Column Names:',df.columns)
print('\n')
print('Unique Values:\n\n',df.nunique())
print('\nNull Values across columns:')
print(df.isna().sum())

In [ ]:
fig,axes=plt.subplots(figsize=[11,8])
fig=sns.heatmap(df.isna(),yticklabels=False ,cmap='YlGnBu',cbar=False )

## We have 10 values missing in Date Added Section. We can handle these missing values by performing MODE operation on Date added and replacing all the missing dates with the most occuring date.

In [ ]:
#Top 10 dates
imp_dates=df['date_added'].value_counts().sort_values(ascending=False).head(10).to_frame()
imp_dates.iplot(kind='bar', layout=({'plot_bgcolor':'black','paper_bgcolor':'black','font_color':'yellow'}),colors='Yellow')

## From the above graph we can observe that maximum number(118) of content gets added to platform on January 1,2020 and also this would be our Mode value as well.Let's go ahead and replace missing values with Mode value

In [ ]:
df['date_added'].fillna(df['date_added'].mode()[0], inplace=True)
print('Null Values in column date_added:',df['date_added'].isna().sum())

In [ ]:
df[df.rating.isna()]

## We can determine the ratings on the basis of values in listed_in column

In [ ]:
rt=df[['rating','listed_in']].value_counts()
rt.iplot(kind='scatter',layout=({'plot_bgcolor':'black','paper_bgcolor':'black','font_color':'yellow'}),colors='Yellow')

In [ ]:
df.loc[ (df['rating'].isna()) & (df['listed_in']=='Movies'), ['rating']] = ['TV-MA']

In [ ]:
df.loc[(df['listed_in'].str.contains('Kids')) & (df['rating'].isna()),['rating']]=['TV-Y7']

In [ ]:
#The highest number of Rating in Dramas is for R, so we would go ahead and assign R as rating for missing values in Drama
df[(df['listed_in']=='Dramas')][['rating','listed_in']].groupby(by='rating').count().sort_values('listed_in',ascending=False)

In [ ]:
df.loc[(df['listed_in'].str.contains('Dramas')) & (df['rating'].isna()),['rating']]=['R']

In [ ]:
#The highest number of Rating in Anime Series, International TV Shows is for TV-14, so we would go ahead and assign TV-14 as rating for missing values in Drama
df[(df['listed_in']=='Anime Series, International TV Shows')][['rating','listed_in']].groupby(by='rating').count().sort_values('listed_in',ascending=False)

In [ ]:
df.loc[(df['listed_in'].str.contains('Anime Series, International TV Shows')) & (df['rating'].isna()),['rating']]=['TV-14']

In [ ]:
print(df.isna().sum())

## Country Analysis for filling the missing country values:

In [ ]:
ca=df[['type','country']].value_counts().head(20).groupby(['country','type']).sum().to_frame().reset_index()
ca.columns=['country','type','Content_Sum']
ca=ca.sort_values(by='Content_Sum',ascending=False)
ca

In [ ]:
fig=px.bar(data_frame=ca,
           x='country',
           y='Content_Sum',
           color='type',
           color_discrete_map={
                     "TV Show":"#ADD8E6",
                     "Movie":"Yellow"} )
fig.update_layout(
    title="Content across countries",
    xaxis_title="Country",
    yaxis_title="Number of Contents Produced",
    legend_title="Type",
    font=dict(family="Arial",size=15, color='yellow'),
    plot_bgcolor='black',
    paper_bgcolor='black')

## From the above Graph we can say that United States is the largest content producer in the world followed by India, and speaking precisely, India majorly produces movies and less of TV shows. We can therfore use this knowledge and try to fill in the missing values in Country Column based on this intel.

In [ ]:
df.loc[(df['country'].isna()) & (df['type']=='TV Show'),['country']]=['United States']

In [ ]:
print(df.isna().sum())

In [ ]:
s=df.loc[(df['country'].isna()) & (df['type']=='Movie')]
s

In [ ]:
def find_nth(string, substring, n):
    if (n == 1):
        return string.find(substring)
    else:
        return string.find(substring, find_nth(string, substring, n - 1) + 1)
    
id_list=[]
for name in s['title']: 
    movies = str(ia.search_movie(name))
    start_index=movies.find(':')+1
    end=find_nth(movies,'[',2)
    movie_id=movies[start_index:end]
    warnings.filterwarnings('ignore')
    #print(movie_id)
    id_list.append(movie_id)


    

In [ ]:

movie_ids=pd.DataFrame(id_list)
movie_ids.columns=['Movie_ID']
movie_ids.replace('[',np.nan, inplace=True)

In [ ]:
movie_ids.index=s.index

In [ ]:
s=pd.concat((s,movie_ids),axis=1)

In [ ]:
from imdb import IMDb
c_list=[]
# create an instance of the IMDb class
ia = IMDb()

# get a movie and print its director(s)
for movie_id in s['Movie_ID']:
    try:
        ids = ia.get_movie(movie_id)
        country=ids['countries']
        #print(country)
        warnings.filterwarnings('ignore')
    except:
        country=np.nan
        #print(country)
        warnings.filterwarnings('ignore')
    
    c_list.append(country)

# show all information that are currently available for a movie
#print(sorted(the_matrix.keys()))

# show all information sets that can be fetched for a movie
#print(ia.get_movie_infoset())

In [ ]:
country_find=pd.DataFrame(c_list, columns=['country'])
country_find.index=s.index
country_find
s['country']=country_find

In [ ]:
s.isnull().sum()

In [ ]:
s['country'].fillna('United States', inplace=True)

In [ ]:
s=s.drop('Movie_ID',axis=1)
s

In [ ]:
#Orginal s frame
s

In [ ]:
#Original df frame
df

In [ ]:
s1=s.copy()

In [ ]:
df1=df.copy()

In [ ]:
new_list=[]
udf=lambda x:x.replace('[','').replace(']','').replace("'","")
for iv in s1.country:
    m=str(iv)
    s=udf(m)
    new_list.append(s)
    #print(s)

In [ ]:
country=pd.DataFrame(new_list,columns=['country'])
country.index=s1.index

In [ ]:
s1['country']=country

In [ ]:
df1.drop(s1.index, inplace=True)

In [ ]:
df1=pd.concat((df1,s1), axis=0)

In [ ]:
df1.sort_index(axis=0, inplace=True)

In [ ]:
df1.isnull().sum()

# Handling Missing Values for Directors:

In [ ]:
df1[df1['director'].isna()]['country'].value_counts().head(10).iplot(kind='bar', layout={'plot_bgcolor':'black','paper_bgcolor':'black','font_color':'yellow'},colors='yellow')

# <h>Top 15 Directors from USA </h>

In [ ]:
show1=df1[df1['country']=='United States']['director'].value_counts().head(15)
show1.iplot(kind='bar', 
            layout=({'plot_bgcolor':'black','paper_bgcolor':'black','font_color':'yellow'}),
            colors='Yellow'
           )

### So for all those cells where directors are missing specifically for USA, we would replace them with these top 15 directors.

In [ ]:
df1[df1['country']=='United States']['director'].value_counts().head(15)

In [ ]:
miss_dir_usa=df1.loc[(df1['director'].isna()) & (df1['country']=='United States')]

In [ ]:
miss_dir_usa.loc[0:3500,'director']='Marcus Raboy'
miss_dir_usa.loc[3501:6000,'director']='Jay Karas'
miss_dir_usa.loc[6001:8000,'director']='Jay Chapman'

#miss_dir_usa.director.value_counts()



In [ ]:
df1=df1.drop(miss_dir_usa.index).reset_index(drop=True)

In [ ]:
df1=pd.concat((df1,miss_dir_usa),axis=0)

In [ ]:
df1.reset_index(drop=True)

## Now we would Handle Missing values for director in United Kingdom:

In [ ]:
show2=df1[df1['country']=='United Kingdom']['director'].value_counts().head(15)
#print(df1[df1['country']=='United Kingdom']['director'].value_counts().head(15))
show2.iplot(kind='bar',layout=({'plot_bgcolor':'black','paper_bgcolor':'black','font_color':'yellow'}), colors='yellow', orientation='v', sortbars=True)

In [ ]:
miss_dir_uk=df1.loc[(df1['director'].isna()) & (df1['country']=='United Kingdom')]

In [ ]:
miss_dir_uk.loc[0:3500,'director']='Edward Cotterill'
miss_dir_uk.loc[3501:5500,'director']='Orlando von Einsiedel'
miss_dir_uk.loc[5501:6729,'director']='Chris Howe'


In [ ]:
df1=df1.drop(miss_dir_uk.index).reset_index(drop=True)
df1=pd.concat((df1,miss_dir_uk),axis=0)
df1.reset_index(drop=True)

## Now for remaining countries we will try to impute the missing values for directors with their mode.

In [ ]:
#Function to impute the director Values:
def best_dir(c,df1):
    dn=df1[df1['country']==c]['director'].value_counts().head(1).to_dict()
    key, value = list(dn.items())[0]
    #print('\n')
    #print('For country {} the best director is {}'.format(c,key))
    df1.loc[(df1['director'].isna()) & (df1['country']==c),['director']]=[key]
    #print('Missing Values for Director are imputed for',c)


In [ ]:
for count in df1.loc[df1['director'].isna()]['country'].unique():
    try:
        best_dir(count,df1)
        warnings.filterwarnings('ignore')
    except:
        pass

In [ ]:
direc=[]
nf=df1[df1['director'].isna()]['country'].str.split(',')
for i in nf:
    #print(i[0])
    direc.append(i[0])    

In [ ]:
sf=pd.DataFrame(direc, columns=['country'])
sf.index=nf.index
sf

In [ ]:
sl=df1[df1['director'].isna()]

In [ ]:
sl.drop('country',axis=1, inplace=True)

In [ ]:
sl['country']=sf

In [ ]:
sl.columns

In [ ]:
sl=sl[['show_id', 'type', 'title', 'director', 'cast','country','date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description']]

In [ ]:
df1.drop(sl.index, inplace=True)

In [ ]:
df1=pd.concat([df1,sl], ignore_index=True)

In [ ]:
for count in df1.loc[df1['director'].isna()]['country'].unique():
    try:
        best_dir(count,df1)
        warnings.filterwarnings('ignore')
    except:
        pass

In [ ]:
df1.loc[df1['director'].isna() & df1['country'].str.startswith('Ukr'),['director']]=['Sergey Loznitsa']
df1.loc[df1['director'].isna() & df1['country'].str.startswith('Fin'),['director']]=['Selma Vilhunen']
df1.loc[df1['director'].isna(),['director']]=['Yorgos Lanthimos']

In [ ]:
cast_miss=df1[df1['cast'].isna()]


In [ ]:
sa=[]
v1=0
v2=100
ln=int(len(df1[df1['cast'].isna()]['title']))
ln

In [ ]:
import imdb
ia = imdb.IMDb()
while(v1<ln):
    for nm in df1[df1['cast'].isna()]['title'][v1:v2]:
        try:
            search_results = ia.search_movie(nm)
            warnings.filterwarnings('ignore')

            if search_results:
                movieID = search_results[0].movieID
                movie = ia.get_movie(movieID)
                if movie:
                    cast = movie.get('cast')
                    topActors = 5
                    for actor in cast[:topActors]:
                        output=(nm,actor['name'])
                        output=list(output)
                        sa.append(output)
                        #print(output)
        except:
            pass
    v1=v1+100
    v2=v2+100

In [ ]:
ex=pd.DataFrame(sa, columns=['Movie','Cast'])
dfn=pd.DataFrame(ex.groupby('Movie')['Cast'].apply(lambda x:', '.join((x.values))))
dfn=dfn.reset_index()
dfn.columns=['title','cast_found']

In [ ]:
new_df=pd.merge(cast_miss,dfn, on='title',how='left')

In [ ]:
new_df.drop('cast',axis=1, inplace=True)

In [ ]:
new_df.rename(columns={'cast_found':'cast'}, inplace=True)

In [ ]:
new_df=new_df[['show_id', 'type', 'title', 'director', 'cast','country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description']]

In [ ]:
df1.drop(cast_miss.index, inplace=True)

In [ ]:
df1=pd.concat([df1,new_df], ignore_index=True)

In [ ]:
df1[df1['cast'].isna()] # Still Missing Cast

In [ ]:
#Function to impute the still missing cast Values:
def best_cast(c,df1):
    db=df1[df1['country']==c]['cast'].value_counts().head(1).to_dict()
    key, value = list(db.items())[0]
    #print('\n')
    #print('For country {} missing cast imputed is {}'.format(c,key))
    df1.loc[(df1['cast'].isna()) & (df1['country']==c),['cast']]=[key]
    #print('Missing Values for Cast are imputed for',c)

In [ ]:
for count in df1.loc[df1['cast'].isna()]['country'].unique():
    try:
        best_cast(count,df1)
    except:
        pass

## From above dataframe we can observe that most of the movies are listed in Documentaries and hence this might be the reason that the cast is still missing for them. So we would go ahead and reaplce the NaN with 'Generic Character' in cast column

In [ ]:
df1.loc[df1['cast'].isna(),['cast']]=['Generic Character']

## - Content According to Ratings

In [ ]:
cr=pd.DataFrame(df1[['type','rating']].value_counts().groupby('rating').sum().sort_values(ascending=False)).reset_index()
cr.columns=['Ratings','Content_Count']

In [ ]:


fig=px.bar(cr,x='Ratings',y='Content_Count' ,color='Ratings')
fig.update_layout(
    title="Count of Content According to Ratings",
    xaxis_title="Ratings",
    yaxis_title="Count",
    legend_title="Rating Type",
    font=dict(family="Arial",size=15))

fig.layout.template = 'plotly_dark'
fig.show()





In [ ]:
fig = px.pie(cr, values='Content_Count', names='Ratings', title='% Distribution by Rating')
fig.layout.template = 'plotly_dark'
fig.show()

## From the above 2 graph, it's pretty evident that most of the content is meant for adult, so there are 2 insights out of this:

- Parents tend to buy other subscriptions for their children, where content is kid/children friendly
- Netflix should increase it's offering in Kids section as well, to increase the market share and coverage.

In [ ]:
fig,axes=plt.subplots(figsize=[15,8])
sns.countplot(x=df1['rating'],
              hue=df1['type'],
              palette="pastel",
              ax=axes,hue_order=['Movie','TV Show']
              
             );


# 10 most frequently appearing directors across the platform, irrespective of country:

In [ ]:
dirct=df1[['director','country']].value_counts().head(10)
fig=dirct.iplot(kind='bar', 
            color='Yellow',
            layout=({'plot_bgcolor':'black', 'paper_bgcolor':'black', 'font_color':'#ADD8E6'}), 
            colors='Yellow',
            title='10 Most Frequently Appearing Directors across the world',
            yTitle='Number of movies Directed',
            xTitle='Directors')



In [ ]:
fig=px.histogram(df['type'],
                 color=df['type'],
                 color_discrete_map={
                     "TV Show":"#ADD8E6",
                     "Movie":"Yellow"},
                 labels=[] 
                
                )

fig.update_layout(
    title="Count of Different Kind of Content on Netflix",
    xaxis_title="Type of Content",
    yaxis_title="Count",
    legend_title="Type",
    font=dict(
        family="Helvetica",
        size=13))

fig.layout.template = 'plotly_dark'

fig.show()

In [ ]:
color=['#ADD8E6','Yellow']
trace=go.Pie(values=df1['type'].value_counts(),
             labels=['Movie','TV Show'],
             marker=dict(colors=color, line=dict(color="#2d2d2d", width=2)),
             title='% Distribution',
             titleposition='bottom center',
             pull=[0.1, 0]
             
            )
fig=go.Figure(data=[trace])
fig.layout.template = 'plotly_white'

iplot(fig)

# Rising Popularity of Netflix across Years:

In [ ]:
df1['date_added']=pd.to_datetime(df1['date_added'])

In [ ]:
df1['Day_Added']=df1['date_added'].dt.day

In [ ]:
df1['Month_Added']=df1['date_added'].dt.month

In [ ]:
df1['Year_Added']=df1['date_added'].dt.year

In [ ]:
ya=df1['Year_Added'].value_counts().to_frame().reset_index().rename(columns={'index':'Year','Year_Added':'Count'})

## Be careful with the release year and year added, release year basically tells the year in which the content is released, year added signifies the year on which the content got added to the platform i.e. Netflix

- Content Addition By Year

In [ ]:
fig=px.bar(data_frame=ya,x='Count',y='Year',
       orientation='h',
       color='Year',
       color_continuous_scale=px.colors.sequential.Darkmint_r)

fig.add_scatter(x=ya['Count'],y=ya['Year'],marker={'color':'yellow'}, showlegend=False)

fig.update_layout(
    title="Addition of Content on Netflix by Year",
    xaxis_title="Count of Contents",
    yaxis_title="Year",
    legend_title="Year",
    font=dict(
        family="Helvetica",
        
        size=13))

fig.layout.template = 'plotly_dark'

fig.iplot()

In [ ]:
cy=df1['release_year'].value_counts().head(20).sort_values(ascending=False).to_frame().reset_index().rename(columns={'index':'Year','release_year':'Count'})


fig=px.bar(data_frame=cy,x='Count',y='Year',
       orientation='h',
       color='Year',
       color_continuous_scale=px.colors.sequential.Darkmint_r)

fig.add_scatter(x=cy['Count'],y=cy['Year'],showlegend=False, marker={'color':'yellow'})

fig.update_layout(
    title="Release of Content on Netflix by Year",
    xaxis_title="Count of Contents",
    yaxis_title="Year",
    legend_title="Year",
    font=dict(
        family="Helvetica",
        size=13))

fig.layout.template = 'plotly_dark'

fig.iplot()


In [ ]:
#This code would take aroud 10 - 15 min to scrap the weekday, since there are large number of records, so be patient
s=[]
for i in range(0,len(df1)):
    a=datetime.strptime(df1['date_added'].astype(str)[i], '%Y-%m-%d').weekday()
    s.append(a)

In [ ]:
import calendar
day=[]
for n in s:
    day_name=calendar.day_name[n]
    day.append(day_name)
   

In [ ]:
day_week=pd.DataFrame(day,columns=['Day_of_Week'])

In [ ]:
df1=pd.concat((df1,day_week),axis=1)

## Let's analyze on which day most of the content gets released.

In [ ]:
rwd=pd.DataFrame(df1.groupby(by='Day_of_Week')['type'].value_counts()).rename(columns={'type':'Count'}).reset_index()
rwd=rwd.sort_values(by='Count',ascending=False)

In [ ]:
#fig,axes=plt.subplots(2,1, figsize=[10,8])
fig=px.bar(data_frame=rwd,
       x='Day_of_Week',
       y='Count', 
       color='type',
       color_discrete_map={'Movie':'#ADD8E6','TV Show':'Yellow'})



fig.update_layout(
    title="Release of Content on Netflix by Weekday",
    yaxis_title="Count of Content",
    xaxis_title="Day of Week",
    legend_title="Type of Content",
    font=dict(
        family="Arial",
        size=13))

fig.layout.template = 'plotly_dark'

fig.iplot()

# Top 10 countries contributing to the Movie Sector:

In [ ]:
top_c=df1[df1['type']=='Movie']['country'].value_counts().head(10).to_frame().reset_index().rename(columns={'index':'country','country':'count'})

color=['#ADD8E6','Yellow','#FFDFD3','#E2F0CB','Green','Orange','Pink','779ECB','Aqua','FF6961']
trace=go.Pie(values=top_c['count'],
             labels=top_c['country'],
             marker=dict(line=dict(color="#2d2d2d", width=2), colors=color)
            )

fig=go.Figure(data=[trace])

fig.layout.template = 'plotly_dark'

fig.update_layout(
    title="% Distribution of Movies by Country",
    legend_title="Countries",
    font=dict(
        family="Arial",
        size=15))

iplot(fig)

## Distribution of TV Show by Country:

In [ ]:
top_s=df1[df1['type']=='TV Show']['country'].value_counts().head(10).to_frame().reset_index().rename(columns={'index':'country','country':'count'})

color=['#ADD8E6','Yellow','#FFDFD3','#E2F0CB','Green','Orange','Pink','779ECB','Aqua','FF6961']
trace=go.Pie(values=top_s['count'],
             labels=top_s['country'],
             marker=dict(line=dict(color="#2d2d2d", width=2), colors=color)
            )

fig=go.Figure(data=[trace])

fig.layout.template = 'plotly_dark'

fig.update_layout(
    title="% Distribution of TV Shows by Country",
    legend_title="Countries",
    font=dict(
        family="Arial",
        size=15))

iplot(fig)

# Let's look at Cast:

# Let's divide our Dataframe into 2 categories: For TV Shows and for Movies

In [ ]:
Movie_df=df1[df1['type']=='Movie']
Show_df=df1[df1['type']=='TV Show']

In [ ]:
n=15
top_cast=Show_df[Show_df['cast']!=np.nan].set_index('title').cast.str.split(', ',expand=True).stack().reset_index(level=1, drop=True).value_counts().head(n).to_frame().reset_index().rename(columns={'index':'cast',0:'count'})
top_cast_m=Movie_df[Movie_df['cast']!=np.nan].set_index('title').cast.str.split(', ',expand=True).stack().reset_index(level=1, drop=True).value_counts().head(n).to_frame().reset_index().rename(columns={'index':'cast',0:'count'})

fig1=px.funnel(data_frame=top_cast,y='cast',x='count',
       orientation='h',
       color='count')


fig2=px.funnel(data_frame=top_cast_m,y='cast',x='count',
       orientation='h',
       color='count')



fig1.update_layout(
    title="Top {} Actor/Actress from TV Shows".format(n),
    yaxis_title="Actors/Actress",
    xaxis_title="Number of Shows Done",
    legend_title="Actors/Actress",
    font=dict(
        family="Arial",
        size=15))


fig2.update_layout(
    title="Top {} Actor/Actress from Movies".format(n),
    yaxis_title="Actors/Actress",
    xaxis_title="Number of Movies Done",
    legend_title="Actors/Actress",
    font=dict(
        family="Arial",
        size=15)     
)


fig1.layout.template = 'plotly_dark'
fig2.layout.template = 'plotly_dark'

fig1.iplot()
fig2.iplot()


# Average Duration around Movies on Netfix

In [ ]:
Movie_df.rename(columns={'duration':'duration_(min)'}, inplace=True)

In [ ]:
Movie_df['duration_(min)']=Movie_df['duration_(min)'].str.replace(' min','').astype(int)

In [ ]:
#fig1=px.histogram(data_frame=Movie_df,x='duration_(min)')

fig1,axes=plt.subplots(figsize=[12,10])
fig1=sns.histplot(x=Movie_df['duration_(min)'], kde=True,fill=True)
fig1=sns.set(style="darkgrid")


# The TOP N Movies:

In [ ]:
N=15
imdb_ratings=pd.read_csv('../input/imdb-extensive-dataset/IMDb ratings.csv',usecols=['weighted_average_vote'])
imdb_titles=pd.read_csv('../input/imdb-extensive-dataset/IMDb movies.csv', usecols=['title','year','genre'])
ratings = pd.DataFrame({'Title':imdb_titles.title,
                    'Release Year':imdb_titles.year,
                    'Rating': imdb_ratings.weighted_average_vote,
                    'Genre':imdb_titles.genre})



ratings=ratings.drop_duplicates(subset=['Title','Release Year','Rating'])
ratings=ratings.dropna()

final_data=Movie_df.merge(ratings,left_on='title',right_on='Title',how='left')
final_data=final_data.sort_values(by='Rating', ascending=False)

top_rated=final_data[0:N]

fig =px.sunburst(
    top_rated,
    path=['title','country'],
    values='Rating',
    color='Rating',
    #color_discrete_sequence=px.colors.sequential.Darkmint_r
    color_continuous_scale=px.colors.sequential.YlGnBu

)
    
fig.update_layout(
    title="Top {} Highly Rated Movies".format(N),
    legend_title="Rating",
    font=dict(
        family="Arial",
        size=13))

fig.layout.template = 'plotly_dark'
fig.show()


# Wordcloud of Most Appearing Indian Actors/Actress:

In [ ]:
most_appear_india=Movie_df[(Movie_df['cast']!=np.nan) & (Movie_df['country']=='India')].set_index('title').cast.str.split(', ',expand=True).stack().reset_index(level=1, drop=True).value_counts().head(50).to_frame().reset_index().rename(columns={'index':'cast',0:'count'})


from wordcloud import WordCloud
wordcloud = WordCloud(background_color="white", width=1400, height=950).generate(" ".join(most_appear_india.cast))

plt.figure(figsize=(14,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.figure(1, figsize=(12, 12))
plt.show()

# If you have liked my work, please do upvote it, since I am a beginner and that would motivate me to keep going :)